In [1]:
# Check out the configurations of Sentinel Hub

#!sentinelhub.config --show

In [2]:
# Update the configurations with the client id, client secret id and instance id from Sentinel Hub
# old secret: m0<l..i&~i_;dz|?C0rC[]JI_5aIy3DL[HEtYCmz
#!sentinelhub.config --sh_client_id '8a419e16-5053-4203-b22c-4e2c448547c1' --sh_client_secret 'm#khV[sw1X8O5tYGwQ&)w?F439(Y#t:hv+ehvje}' --instance_id '37abf98d-a04e-40df-9a9b-6b58fb18ee6e'
#config.save()


In [41]:
# Create a config instance from Sentinel Hub

from sentinelhub import SHConfig
config = SHConfig()
config

SHConfig(
  instance_id='37abf98d-a04e-40df-9a9b-6b58fb18ee6e',
  sh_client_id='8a419e16-5053-4203-b22c-4e2c448547c1',
  sh_client_secret='m#khV[sw1X8O5tYGwQ&)w?F439(Y#t:hv+ehvje}',
  sh_base_url='https://services.sentinel-hub.com',
  sh_auth_base_url='https://services.sentinel-hub.com',
  geopedia_wms_url='https://service.geopedia.world',
  geopedia_rest_url='https://www.geopedia.world/rest',
  aws_access_key_id='',
  aws_secret_access_key='',
  aws_session_token='',
  aws_metadata_url='https://roda.sentinel-hub.com',
  aws_s3_l1c_bucket='sentinel-s2-l1c',
  aws_s3_l2a_bucket='sentinel-s2-l2a',
  opensearch_url='http://opensearch.sentinel-hub.com/resto/api/collections/Sentinel2',
  max_wfs_records_per_query=100,
  max_opensearch_records_per_query=500,
  max_download_attempts=4,
  download_sleep_time=5.0,
  download_timeout_seconds=120.0,
  number_of_download_processes=1
)

In [42]:
# Logging config

import logging, sys

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

fhandler = logging.FileHandler(filename='api_log.log', mode='a')
strhandler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.addHandler(strhandler)

In [43]:
# IPython magic functions

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [44]:
# Import packages

import datetime, os, sys

import matplotlib.pyplot as plt
import numpy as np
from shapely import geometry
import pandas as pd
from datetime import datetime
import time, ast
from dateutil.relativedelta import relativedelta

from sentinelhub import (
    CRS,
    BBox,
    Geometry,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
    get_image_dimension
)

In [35]:
# Import CSV file with data about the Area of Interest (AoI)

try:
    aoi_csv = pd.read_csv('../aoi_data.csv', header=0)
    logger.info('CSV data imported')
    print(aoi_csv.head())
except Exception as e:
    logger.error('Error importing the csv file: ', e)
    

CSV data imported
CSV data imported
                                    Area of interest End of fire  \
0  EMSR175_01CALHETA_GRADING_OVERVIEW-MONIT02_v1_...  2016-08-16   
1       EMSR175_02FUNCHAL_GRADING_OVERVIEW_v1_vector  2016-08-11   
2  EMSR210_05ELCAMPILLOOVERVIEW_02GRADING_MAP_v1_...  2017-07-08   
3           EMSR211_02SONEJA_02GRADING_MAP_v1_vector  2017-07-08   
4      EMSR213_01VESUVIO_02GRADING_MONIT01_v2_vector  2017-07-16   

                                    Bbox coordinates Polygon coordinates  \
0  [-17.2144494574379507,32.6995227836476303,-17....                 NaN   
1  [-17.0297773356982809,32.6313310944295978,-16....                 NaN   
2  [-6.7200183960538604,37.6850935359744525,-6.59...                 NaN   
3  [-0.4849453245600591,39.7814226710891532,-0.42...                 NaN   
4  [14.3658109057387406,40.7643571090310814,14.47...                 NaN   

   width  height            Location        Code  QA ok=1; nok=0  \
0  653.0   512.0             C

INFO:__main__:CSV data imported


In [46]:
# Import CSV file with data about the Area of Interest (AoI)

try:
    aoi_csv = pd.read_csv('../aoi_data_add.csv', header=0)
    logger.info('CSV data imported')
    print(aoi_csv.head())
except Exception as e:
    logger.error('Error importing the csv file: ', e)

CSV data imported
CSV data imported
CSV data imported
                               Area of interest End of fire  \
0   EMSR344_01CALENZANA_02GRADING_MAP_v2_vector  2019-03-28   
1  EMSR428_AOI01_GRA_PRODUCT_r1_RTP01_v2_vector  2020-02-27   
2  EMSR506_AOI01_GRA_PRODUCT_r1_RTP01_v1_vector  2021-04-09   
3  EMSR620_AOI01_GRA_PRODUCT_r1_RTP01_v1_vector  2022-08-15   
4  EMSR638_AOI01_GRA_PRODUCT_r1_RTP01_v1_vector  2022-11-11   

                                    Bbox coordinates  Polygon coordinates  \
0  [8.7015078210000638,42.4010933590000718,8.9364...                  NaN   
1  [-15.7844236549999604,27.9064536160000785,-15....                  NaN   
2  [-100.5435486129999845,25.3073887350000746,-10...                  NaN   
3  [-4.9067605349999326,42.9342820860000529,-4.80...                  NaN   
4  [25.2246621480000499,41.1778268960000560,25.37...                  NaN   

   width  height                   Location           Code  QA ok=1; nok=0  \
0    NaN     NaN          

INFO:__main__:CSV data imported


In [47]:
# Parse the coordinates of boxes
def parse_coord_bbox(str_coordinates):
    try:
        coordinates = str_coordinates[1:-1].split(',')
        coord_list = [float(coord) for coord in coordinates]
        return coord_list
    except Exception as e:
        logger.error('Error occured parsing the box coordinates: ', e)

In [48]:
# Parse the coordinates of polygones
def parse_coord_poly(str_coordinates):
    try:
        coord_str = str_coordinates[1:-1].replace(' ','')
        coord_list = ast.literal_eval(coord_str)
        return coord_list
    except Exception as e:
        logger.error('Error occured parsing the poly coordinates: ', e)

In [49]:
# Determine width & height of image

def get_width_height(bbox_coordinates: BBox):
    height = 512
    width = get_image_dimension(bbox_coordinates, height=512)
    if width < height:
        width = 512
        height = get_image_dimension(bbox_coordinates, width=512)
    return width, height

In [51]:
# Retrieve satellite images providing boxes as AoI delimitations

try:
    aoi_bbox = aoi_csv.loc[aoi_csv['Bbox coordinates'].notna()]
    aoi_bbox.reset_index(inplace=True)
    
    start = time.time()

    for index, row in aoi_bbox.iterrows():
        #logger.debug('Processing AOI with index: ', index)
        time_from = row['End of fire']
        date_delta = datetime.strptime(time_from, '%Y-%m-%d').date() + relativedelta(weeks=+2)
        time_until = date_delta.strftime('%Y-%m-%d')
        aoi_name = row['Location']
        folder_name = row['Area of interest']
        aoi_coords_wgs84 = parse_coord_bbox(row['Bbox coordinates'])
        aoi_bbox_coord = BBox(bbox=aoi_coords_wgs84, crs=CRS.WGS84)
        width, height = get_width_height(aoi_bbox_coord)
        aoi_bbox.loc[index:index,'width'] = width
        aoi_bbox.loc[index:index,'height'] = height
        
        evalscript_all_bands = """
        //VERSION=3

        function setup() {
        return {
            input: ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B10", "BQA", "dataMask"],
            output: {
            id: "reflectance",
            bands: 11,
            sampleType: SampleType.UINT16 //floating point values are automatically rounded to the nearest integer by the service.
            }
        };
        }
        function evaluatePixel(sample, scenes, inputMetadata, customData, outputMetadata) {
        return [sample.B01, sample.B02, sample.B03, sample.B04, sample.B05, sample.B06, sample.B07, sample.B10, sample.BQA, sample.dataMask]
        }
        """
        request_all_bands = SentinelHubRequest(
            evalscript=evalscript_all_bands,
            input_data=[
                SentinelHubRequest.input_data(
                    data_collection=DataCollection.LANDSAT_OT_L2,
                    time_interval=(time_from, time_until),
                    mosaicking_order=MosaickingOrder.LEAST_CC,
                )
            ],
            responses=[SentinelHubRequest.output_response('reflectance', MimeType.TIFF)],
            bbox=aoi_bbox_coord,
            size=(width,height),
            data_folder=f'../Data/Add_Landsat_images_level2_2W/{folder_name}',
            config=config,
        )
        request_all_bands.save_data()
        
    end = time.time()
    print(f'Execution time: {end - start:.2f} seconds')
    #aoi_bbox.to_csv(path_or_buf='aoi_bbox_w_h.csv', index=False)
except Exception as e:
    #logger.error(f'Error occured while requesting satellite images: ', e)
    print(e)
    

Execution time: 13.71 seconds


In [13]:
# Retrieve satellite images providing polygons as AoI delimitations
# mosaicking_order=MosaickingOrder.LEAST_CC,


try:
    aoi_poly = aoi_csv.loc[aoi_csv['Polygon coordinates'].notna()]
    aoi_poly.reset_index(inplace=True)

    start = time.time()

    for index, row in aoi_poly.iterrows():
        time_from = row['End of fire']
        date_delta = datetime.strptime(row['End of fire'], '%Y-%m-%d').date() + relativedelta(months=+2)
        time_until = date_delta.strftime('%Y-%m-%d')
        aoi_name = row['Location']
        folder_name = row['Area of interest']
        aoi_polygon = Geometry(geometry.Polygon(parse_coor_poly(row['Polygon coordinates'])), '4326')
        aoi_coords_wgs84 = parse_coord_bbox(row['Bbox coordinates'])
        aoi_bbox_coord = BBox(bbox=aoi_coords_wgs84, crs=CRS.WGS84)
        width, height = get_width_height(aoi_bbox_coord)
        aoi_poly.loc[index:index,'width'] = width
        aoi_poly.loc[index:index,'height'] = height

        evalscript_all_bands = """
        //VERSION=3

        function setup() {
        return {
            input: ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B10", "B11", "BQA", "dataMask"],
            output: {
            id: "reflectance",
            bands: 13,
            sampleType: SampleType.UINT16 //floating point values are automatically rounded to the nearest integer by the service.
            }
        };
        }
        function evaluatePixel(sample, scenes, inputMetadata, customData, outputMetadata) {
        return [sample.B01, sample.B02, sample.B03, sample.B04, sample.B05, sample.B06, sample.B07, sample.B10, sample.B11, sample.BQA, sample.dataMask, outputMetadata]
        }
        """
        request_all_bands = SentinelHubRequest(
            evalscript=evalscript_all_bands,
            input_data=[
                SentinelHubRequest.input_data(
                    data_collection=DataCollection.LANDSAT_OT_L2,
                    time_interval=(time_from, time_until),
                    mosaicking_order=MosaickingOrder.LEAST_RECENT,
                    maxcc=0.3
                )
            ],
            responses=[SentinelHubRequest.output_response('reflectance', MimeType.TIFF)],
            geometry=aoi_polygon,
            size=(width, height),
            data_folder=f'../Data/Landsat_images_level2_2M/{folder_name}',
            config=config,
        )
        request_all_bands.save_data()
        
    end = time.time()
    print(f'Execution time: {end - start:.2f} seconds')
    #aoi_poly.to_csv(path_or_buf='aoi_poly_w_h.csv', index=False)
except Exception as e:
    print('Error: ', e)
    logger.error('Error occured while requesting satellite images: ', e)
    

Execution time: 570.78 seconds


In [39]:
# Save width/height in from a CSV

def save_width_height_all(csv)
    for i, row in aoi_csv.iterrows():
        aoi_coords_wgs84 = parse_coord_bbox(row['Bbox coordinates'])
        aoi_bbox_coord = BBox(bbox=aoi_coords_wgs84, crs=CRS.WGS84)
        width, height = get_width_height(aoi_bbox_coord)
        aoi_csv.loc[i:i,'width'] = width
        aoi_csv.loc[i:i,'height'] = height  
    aoi_csv.to_csv(path_or_buf='aoi_w_h.csv', index=False)
    return